In [1]:
from prediction import TokenPredictor
from llm_compressor import LLMCompressor
import numpy as np
# This script initializes a TokenPredictor with a specified data path.
data_path = '../data/text8'
token_predictor = TokenPredictor(data_path)

/var/lit2425/humanize/summer-offsite/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Starting tokenization...


Token indices sequence length is longer than the specified maximum sequence length for this model (19428084 > 131072). Running this sequence through the model will result in indexing errors


Tokenization complete. Total number of tokens: 19428084
Number of distinct tokens: 35844


In [9]:
data = token_predictor.get_data_tokens()[:10]
prompt_tokens = []
import numpy as np

llm_compressor = LLMCompressor()

for i in range(1, len(data)):
    # pop the first token if the prompt is too long
    if len(prompt_tokens) > 130000:
        prompt_tokens.pop()
    # add the next token to the prompt
    prompt_tokens.append(data[i-1])
    # print the current status in one line
    # print(f"\rProcessing token {i}/{len(data)}", end='')
    # print(f"\nprompt_tokens: {prompt_tokens}")
    print(f"Prompt Text: {token_predictor.detokenize(prompt_tokens)}")
    print(f"Next token will be {data[i]}")
    next_token_index, probs_values = token_predictor.get_ids_probs(prompt_tokens, data[i])
    probs_values = np.array(probs_values)
    llm_compressor.next_token(next_token_index, probs_values)
    print(f"Index of next token in top ids: {next_token_index}")
    print(f"probabilities of the correct token: {probs_values[next_token_index]}")
    print(f"Actial token: {token_predictor.get_token_by_id(next_token_index)}")

bit_string = llm_compressor.compress()

Prompt Text:  anarch
Next token will be 2142
Index of next token in top ids: 34544
probabilities of the correct token: 0.005449335090816021
Actial token: 94296
Prompt Text:  anarchism
Next token will be 43753
Index of next token in top ids: 16807
probabilities of the correct token: 0.00030313548631966114
Actial token: 37330
Prompt Text:  anarchism originated
Next token will be 438
Index of next token in top ids: 35684
probabilities of the correct token: 0.053930994123220444
Actial token: 98592
Prompt Text:  anarchism originated as
Next token will be 264
Index of next token in top ids: 35812
probabilities of the correct token: 0.7700403928756714
Actial token: 99113
Prompt Text:  anarchism originated as a
Next token will be 4647
Index of next token in top ids: 33023
probabilities of the correct token: 0.0007639548857696354
Actial token: 88845
Prompt Text:  anarchism originated as a term
Next token will be 315
Index of next token in top ids: 35769
probabilities of the correct token: 0.134

In [6]:
# decompress
from llm_compressor import LLMDecompressor
decompressor = LLMDecompressor(bit_string)
decompress_prompt_tokens = []
decompress_prompt_tokens.append(data[0])
for i in range(1, len(data)):
    if len(decompress_prompt_tokens) > 130000:
        decompress_prompt_tokens.pop()
    print(f"decompress_prompt_tokens: {decompress_prompt_tokens}")
    _, _, probs_values = token_predictor.get_token_info(decompress_prompt_tokens)
    probs_values = np.array(probs_values)
    # if i == 1:
    #     print(probs_values[:10])
    token_idx = decompressor.decompress(probs_values)
    # print(token_idx, data[i], token_predictor.get_token_by_id(token_idx))
    decompress_prompt_tokens.append(token_predictor.get_token_by_id(token_idx))
    # print(f"\rDecompressing token {i+1}/{len(data)}", end='')
    print(f"decompress_prompt_tokens: {decompress_prompt_tokens}")
    print(f"Original token: {data[:i+1]}")
    # assert decompress_prompt_tokens == data[:i+1]

decompress_prompt_tokens: [43465]
decompress_prompt_tokens: [43465, 94296]
Original token: [43465, 2142]
decompress_prompt_tokens: [43465, 94296]
decompress_prompt_tokens: [43465, 94296, 37330]
Original token: [43465, 2142, 43753]
decompress_prompt_tokens: [43465, 94296, 37330]
decompress_prompt_tokens: [43465, 94296, 37330, 98592]
Original token: [43465, 2142, 43753, 438]
decompress_prompt_tokens: [43465, 94296, 37330, 98592]
decompress_prompt_tokens: [43465, 94296, 37330, 98592, 99113]
Original token: [43465, 2142, 43753, 438, 264]
decompress_prompt_tokens: [43465, 94296, 37330, 98592, 99113]
decompress_prompt_tokens: [43465, 94296, 37330, 98592, 99113, 88845]
Original token: [43465, 2142, 43753, 438, 264, 4647]
decompress_prompt_tokens: [43465, 94296, 37330, 98592, 99113, 88845]
decompress_prompt_tokens: [43465, 94296, 37330, 98592, 99113, 88845, 98943]
Original token: [43465, 2142, 43753, 438, 264, 4647, 315]
decompress_prompt_tokens: [43465, 94296, 37330, 98592, 99113, 88845, 9894

KeyboardInterrupt: 

In [ ]:
len(bit_string)  # Length of the compressed bit string

1496

In [ ]:
output_string = token_predictor.detokenize(data)
print(len(output_string))  # Length of the original text

533


In [ ]:
1496/(533*8)

0.350844277673546